# Import Packages

In [72]:
import pandas as pd
from sklearn.cluster import DBSCAN
import prince
from sklearn.preprocessing import StandardScaler
import gower
from sklearn.cluster import KMeans

# Load Data

In [80]:
df = pd.read_csv("../data/prepped_data.csv", low_memory=False, index_col=0).drop_duplicates()

df = df[df["first_data_year"] >= 2021].head(5000)

columns_clustering = ['last_customer_age', 'last_accident_free_years', 'last_car_value', 'last_age_car', "last_postcode", "last_fuel_type", "nr_years", "last_premium", 'last_sales_channel', 'pc4', 'median_income_household', 'density', 'perc_others_ppl']

# Apply Clustering

In [84]:
dist_matrix = gower.gower_matrix(df[columns_clustering])

cluster = KMeans(n_clusters=3, random_state=0, n_init="auto").fit(dist_matrix)

df["cluster"] = cluster.labels_

display(
    df
    .groupby("cluster")
    .agg(
        income=pd.NamedAgg(column="median_income_household", aggfunc="mean"),
        perc_low_income=pd.NamedAgg(column="perc_low_income", aggfunc="mean"),
        perc_high_income=pd.NamedAgg(column="perc_high_income", aggfunc="mean"),
        density=pd.NamedAgg(column="density", aggfunc="mean"),
        household_size=pd.NamedAgg(column="household_size", aggfunc="mean"),
        welcome_discount=pd.NamedAgg(column="welcome_discount", aggfunc="mean"),
        churn=pd.NamedAgg(column="churn", aggfunc="mean"),
        last_customer_age=pd.NamedAgg(column="last_customer_age", aggfunc="mean"),
        last_car_value=pd.NamedAgg(column="last_car_value", aggfunc="mean"),
        # perc_others_ppl=pd.NamedAgg(column="perc_others_ppl", aggfunc="mean"),
    )
)

,income,perc_low_income,perc_high_income,density,household_size,welcome_discount,churn,last_customer_age,last_car_value
cluster,,,,,,,,,
0,33251.653196,36.941587,22.710580,1910.099192,2.251212,0.044035,0.109478,49.691403,330422.951359
1,32276.230769,36.918577,20.009308,1073.104615,2.237423,0.039511,0.094615,51.356538,367588.191385
2,29644.947064,46.450529,16.895380,3060.218479,2.125987,0.127101,0.208855,44.684312,467539.752839
